# Traveling Salesman Problem

We are given a [complete graph](https://en.wikipedia.org/wiki/Complete_graph) which means that every pair of distinct vertices is connected by a unique edge. Each edge has traveling cost associated with it. We need to find a shortest route that goes throught all the nodes and comes back to the original node.

## Representation

We can represent traveling costs as a simple matrix.

In [1]:
i = 'i'
m =  [
    [i, 3, 1, 8],
    [3, i, 4, 4],
    [1, 4, i, 7],
    [8, 4, 7, i],
]

This represents 4 nodes (0 through 3) where cost of traveling from 0 to 3 is

In [2]:
m[0][3]

8

## Brute force solution

In [3]:
from itertools import permutations


def brute_force(matrix):
    current_min = pow(10,10) # should be at least bigger than min route
    current_route = []
    for nodes in permutations(range(len(matrix))):
        # we are going from node a to node b in each step
        a = list(nodes)
        b = a[1:] + [a[0]]

        # calculate the route cost
        route_cost = 0
        for ii, jj in zip(a, b):
            route_cost += matrix[ii][jj]

        # check if we got new min route
        if route_cost < current_min:
            current_min = route_cost
            current_route = nodes
    
    return current_route, current_min

In [4]:
route, distance = brute_force(m)

In [5]:
print('Found route:', route, 'with distance:', distance)

Found route: (0, 1, 3, 2) with distance: 15
